In [1]:
# Copyright 2021 DeepMind Technologies Limited
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title Import
from typing import Union

# Haiku permet aux utilisateurs d'utiliser des modèles de POO tout en permettant un accès complet aux transformations de fonctions pures de JAX pour les reseaux neuronaux.
import haiku as hk
# JAX est concue pour le calcul numérique à haute performance, notamment pour la recherche en apprentissage automatique.
import jax
import jax.numpy as jnp
# NumPy ajoute un support pour les grands tableaux multidimensionnels et les matrices, ainsi qu'une grande collection de fonctions mathématiques pour opérer sur ces tableaux.
import numpy as np
# Pickle est un module permettant la (dé)sérialisation
import pickle

import perceiver
import position_encoding
import io_processors
import bytes_tokenizer

In [3]:
#@title Load parameters from checkpoint
#!wget -O language_perceiver_io_bytes.pickle https://storage.googleapis.com/perceiver_io/language_perceiver_io_bytes.pickle

# On deserialise les valeurs des hyperparametres du modele.
with open("language_perceiver_io_bytes.pickle", "rb") as f:
  params = pickle.loads(f.read())

In [4]:
#@title Model config
D_MODEL = 768
D_LATENTS = 1280
MAX_SEQ_LEN = 2048

encoder_config = dict(
    num_self_attends_per_block=26,
    num_blocks=1,
    z_index_dim=256,
    num_z_channels=D_LATENTS,
    num_self_attend_heads=8,
    num_cross_attend_heads=8,
    qk_channels=8 * 32,
    v_channels=D_LATENTS,
    use_query_residual=True,
    cross_attend_widening_factor=1,
    self_attend_widening_factor=1)

decoder_config = dict(
    output_num_channels=D_LATENTS,
    position_encoding_type='trainable',
    output_index_dims=MAX_SEQ_LEN,
    num_z_channels=D_LATENTS,
    qk_channels=8 * 32,
    v_channels=D_MODEL,
    num_heads=8,
    final_project=False,
    use_query_residual=False,
    trainable_position_encoding_kwargs=dict(num_channels=D_MODEL))

# The tokenizer is just UTF-8 encoding (with an offset)
tokenizer = bytes_tokenizer.BytesTokenizer()

In [27]:
#@title Decoding Perceiver Model
def apply_perceiver(
    inputs: jnp.ndarray, input_mask: jnp.ndarray) -> jnp.ndarray:
  """Runs a forward pass on the Perceiver.

  Args:
    inputs: input bytes, an int array of shape [B, T]
    input_mask: Array of shape indicating which entries are valid and which are
      masked. A truthy value indicates that the entry is valid.

  Returns:
    The output logits, an array of shape [B, T, vocab_size].
  """
  assert inputs.shape[1] == MAX_SEQ_LEN

  # https://gdcoder.com/what-is-an-embedding-layer/
  # Une couche d'integration convertie l'input en un ensemble de vecteurs
  # d'integration dont les tailles sont optimisees pour le calcul

  # Creation de la couche d'integration selon le nombre de mots distincts dans
  # le training set et la dimension voulue des vecteurs d'integrations
  embedding_layer = hk.Embed(
      vocab_size=tokenizer.vocab_size,
      embed_dim=D_MODEL)
  # Conversion de l'input pour rentrer dans la 1ere couche (couche d'integration)
  embedded_inputs = embedding_layer(inputs)

  # Taille de la conversion de l'input
  batch_size = embedded_inputs.shape[0]

  # Parametres de construction des informations entrainables a propos de
  # la position des mots dans la phrase
  input_pos_encoding = perceiver.position_encoding.TrainablePositionEncoding(
      index_dim=MAX_SEQ_LEN, num_channels=D_MODEL)
  #print(input_pos_encoding(batch_size))
  # Ajout des informations de position des mots a l'input
  embedded_inputs = embedded_inputs + input_pos_encoding(batch_size)
  # Initialisation du PerceiverIO
  perceiver_mod = perceiver.Perceiver(
      encoder=perceiver.PerceiverEncoder(**encoder_config),
      decoder=perceiver.BasicDecoder(**decoder_config))
  # Stockage dans output du resultat de l'execution du PerceiverIO a partir de la 1ere couche
  output_embeddings = perceiver_mod(
      embedded_inputs, is_training=False, input_mask=input_mask, query_mask=input_mask)

  # Redimensionnement et decodage de l'output
  logits = io_processors.EmbeddingDecoder(
      embedding_matrix=embedding_layer.embeddings)(output_embeddings)
  return logits

# La transformation de la fonction lui permettra plus tard de lui
# passer des parametres pre-enregistres
# input_pos_encoding sera notamment ecrase par celui pre-enregistre
apply_perceiver = hk.transform(apply_perceiver).apply

In [6]:
# Input sous format initial
input_str = "This is the best thing that happened to me!"
# On marque les coordonnees a predire
input_tokens_positions = [
    input_str.index(" is"),input_str.index("the best")
]
# vectorisation en bytes de l'input, sous forme d'entiers
input_tokens = tokenizer.to_int(input_str)

# Note that the model performs much better if the masked chunk
# starts with a space.
# On masque la partie de l'input a predire avec une constante de tokenizer
input_tokens[input_tokens_positions[0]:input_tokens_positions[1]] = tokenizer.mask_token
print("Tokenized string without masked bytes:")
print(tokenizer.to_string(input_tokens))

Tokenized string without masked bytes:
Thisthe best thing that happened to me!


In [32]:
#@title Pad and reshape inputs
#print(input_tokens)
# inputs est le tableau d'entiers avec une dimension supplementaire
# c'est comme si on mettait le tableau inputs a l'interieur d'un nouveau tableau
inputs = input_tokens[None]
#print(inputs)
# input_mask est l'equivalent unitaire de inputs :
# pour toutes les valeurs d'inputs, on met un 1
input_mask = np.ones_like(inputs)
#print(input_mask)

# Agrandit les tableaux inputs et input_mask pour qu'ils aient
# une taille de max_sequence_length, et les remplit avec des 0
def pad(max_sequence_length: int, inputs, input_mask):
  input_len = inputs.shape[1]
  assert input_len <= max_sequence_length
  pad_len = max_sequence_length - input_len
  padded_inputs = np.pad(
      inputs,
      pad_width=((0, 0), (0, pad_len)),
      constant_values=tokenizer.pad_token)
  padded_mask = np.pad(
      input_mask,
      pad_width=((0, 0), (0, pad_len)),
      constant_values=0)
  return padded_inputs, padded_mask

inputs, input_mask = pad(MAX_SEQ_LEN, inputs, input_mask)
#print(inputs)
#print(input_mask)

In [30]:
rng = jax.random.PRNGKey(1)  # Unused
#print(params)
out = apply_perceiver(params, rng=rng, inputs=inputs, input_mask=input_mask)

# la prediction sous forme d'entiers est constituee des valeurs maximales
# aux coordonnees du masque sur la 1ere dimension de out
#print(out)
masked_tokens_predictions = out[0, input_tokens_positions[0]:input_tokens_positions[1]].argmax(axis=-1)
print("Greedy predictions:")
print(masked_tokens_predictions) #predictions sous formes de caracteres
print()
print("Predicted string:")
print(tokenizer.to_string(masked_tokens_predictions))

FlatMapping({
  'trainable_position_encoding': FlatMapping({
                                   'pos_embs': array([[ 0.45652774, -0.05273699,  0.09694741, ...,  0.08061957,
                                                        0.0847721 , -0.03647762],
                                                      [ 0.53588545, -0.6020879 , -0.52604985, ...,  0.29653946,
                                                        0.15554446,  0.01623693],
                                                      [ 0.4893116 , -0.18050799, -0.7247568 , ...,  0.03686034,
                                                       -0.13996553,  0.04942406],
                                                      ...,
                                                      [-0.19481438, -0.4088347 ,  0.29001424, ..., -0.5040369 ,
                                                        0.17912908,  0.02215274],
                                                      [-0.16852173, -0.19396524,  0.03744941, ..., -0.07